In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import tensorflow as tf
# model = __import__("model")
from cv2 import cv2
import sys
import os
import time
import math
import numpy as np
from math import sin , cos
from tqdm import tqdm

In [ ]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My\ Drive/Research/

/content/drive/My Drive/Research


# Preprocess dataset video to frames

In [ ]:
# The max number of frames to be processed, 
# and the number of frames already processed
NFRAMES = 1000
vid_dir = "./Dataset/"
# TODO: Do not hard code the number of dataset files and filename
for i in tqdm(range(1,12)):
  
  # Open the video file
  vid_path = f"out-video-{i}"
  vid_ext = ".avi"
  path = vid_dir + vid_path + vid_ext
  assert os.path.isfile(path)
  cap = cv2.VideoCapture(path)

  curFrame = 0

  while (cap.isOpened()):
    if curFrame < NFRAMES:
      cam_start = time.time()
      ret, img = cap.read()

      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

      #Preprocess the input frame
      img = cv2.resize(img, (200, 66))
      img = img / 255.

      curFrame += 1

      filename = f"train_images/{vid_path}_frame_{curFrame}.jpg" 
      frame = img
      # print(vid_dir + filename)
      cv2.imwrite(vid_dir + filename, frame)

    else:
      break

cap.release()
cv2.destroyAllWindows()	



# Define model

In [ ]:
# To make sure there is no inqeuivalent output from tf.nn.conv2d and keras.layers.Conv2D
# ref:https://stackoverflow.com/questions/61087933/inequivalent-output-from-tf-nn-conv2d-and-keras-layers-conv2d
tf.keras.backend.set_floatx('float32')

# Model 5 conv 3fc
img_height = 66
img_width =  200
img_channels = 3
input_shape = [ img_height, img_width, img_channels]

#Input layer
input_x = tf.keras.layers.Conv2D(filters=3, kernel_size=[5,5], input_shape=input_shape,activation='relu',  padding="same")
input_max = tf.keras.layers.MaxPooling2D(5,2)

# 1st convolution layer
h_conv1 = tf.keras.layers.Conv2D(
                  filters=24,
                  kernel_size=[5,5],
                  input_shape= input_shape,
                  activation='relu',  padding="same")
h_max1 = tf.keras.layers.MaxPooling2D(5,2)

# 2nd convolution layer
h_conv2 = tf.keras.layers.Conv2D(
                  filters=36,
                  kernel_size=[5,5],
                  activation='relu', padding="same") 
h_max2 = tf.keras.layers.MaxPooling2D(5,2)

# 3rd convolution layer
h_conv3 = tf.keras.layers.Conv2D(
                  filters=48,
                  kernel_size=[5,5],
                  activation='relu', padding="same") 
h_max3 = tf.keras.layers.MaxPooling2D(3,1)

# 4th convolution layer
h_conv4 = tf.keras.layers.Conv2D(
                  filters=64,
                  kernel_size=[3,3],
                  activation='relu', padding="same") 
h_max4 = tf.keras.layers.MaxPooling2D(3,1)

# 5th convolution layer
h_conv5 = tf.keras.layers.Conv2D(
                  filters=64,
                  kernel_size=[3,3],
                  activation='relu', padding="same") 

h_conv5_flat = tf.keras.layers.Flatten() #1152
h_fc2 = tf.keras.layers.Dense(100, activation='relu')
h_fc3 = tf.keras.layers.Dense(50, activation='relu')
h_fc4 = tf.keras.layers.Dense(10, activation='relu')
# missing atan and multiply
y = tf.keras.layers.Dense(1, activation='softmax')
                

In [ ]:
# https://stackoverflow.com/questions/42785026/tf-nn-conv2d-vs-tf-layers-conv2d
model = tf.keras.models.Sequential([
            input_x,
            input_max,
            h_conv1,
            h_max1 ,
            h_conv2,
            h_max2,
            h_conv3,
            h_max3,
            h_conv4,
            h_max4,
            h_conv5,
            h_conv5_flat,
            h_fc2,
            h_fc3,
            h_fc4,
            y,
])
print(model.summary())

Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_92 (Conv2D)           (None, 66, 200, 3)        228       
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 31, 98, 3)         0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 14, 47, 24)        0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 5, 22, 36)         0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 5, 22, 48)       